I just archived the code I used to find the best Arima params- this notebook is not useful now

Drop test columns which have nans : merge on teh right then resplit

In [ ]:
df_spring = df_spring.merge(df_spring_test, left_index=True, right_index=True)
df_summer = df_summer.merge(df_spring_test, left_index=True, right_index=True)
df_fall = df_fall.merge(df_spring_test, left_index=True, right_index=True)

In [ ]:
df_spring = df_spring.dropna(how="any")
df_summer = df_summer.dropna(how="any")
df_fall = df_fall.dropna(how="any")

Redo the split

In [ ]:
df_spring_test = df_spring.iloc[:,-1]
df_summer_test = df_summer.iloc[:,-1]
df_fall_test = df_fall.iloc[:,-1]

df_spring = df_spring.iloc[:,:-1]
df_summer = df_summer.iloc[:,:-1]
df_fall = df_fall.iloc[:,:-1]

Now let's train the modell

In [ ]:
print( len(df_spring.columns), len(df_summer.columns),len(df_fall.columns))

9 9 8


## Model Training

In [ ]:
df_spring.values.flatten()

array([3.136, 3.21 , 3.086, ..., 4.   , 4.   , 4.   ])

Rename indices to be in order

In [ ]:
df_spring_train = df_spring.copy()
df_summer_train = df_summer.copy()
df_fall_train = df_fall.copy()

df_spring_train.columns = [1,2,3,4,5,6,7,8,9]
df_summer_train.columns = [1,2,3,4,5,6,7,8,9]
df_fall_train.columns = [1,2,3,4,5,6,7,8]

df_spring_validation = df_spring_test.copy()
df_summer_validation = df_summer_test.copy()
df_fall_validation = df_fall_test.copy()

df_spring_validation.columns = [10]
df_summer_validation.columns = [10]
df_fall_validation.columns = [9]


Get list of courses and put everyting into list

In [ ]:
spring_courses = df_spring_train.index.to_list()
summer_courses = df_summer_train.index.to_list()
fall_courses = df_fall_train.index.to_list()

dfs = [df_spring_train, df_summer_train, df_fall_train]
tests = [df_spring_validation, df_summer_validation, df_fall_validation]
courses = [spring_courses, summer_courses, fall_courses]
prediction_indices = [10,10,9]

Get it working for a single course

In [ ]:
semester = 0 # 0 = spring, 1 = summer, 2 = fall

df = dfs[semester]
test = tests[semester]
course_list = courses[semester]
prediction_index = prediction_indices[semester]


# specify parameters
p,d,q = 1,1,2

# one for now
course_name = course_list[0]
# get all grades for this course and put into a dataframe
train_row = df.loc[course_name].tolist()

# ARIMA
model = ARIMA(train_row, order=(1,1,2)).fit()

# get predictions for all periods
pred1 = model.predict()

# since ARIMA starts at 1 pred1[0] is not calculated, use real grades as the starting point (for graph)
pred1[0] = train_row[0]

# next period's prediction
prediction = model.predict(prediction_index)

print("train predictions", pred1, " \n test predictions", prediction)

print("Train original values", train_row, "\n Test original values", test.loc[course_name].tolist())


print("MSE train data: ", mean_squared_error(train_row, pred1))
print("MSE test data: ", mean_squared_error([test.loc[course_name]], prediction))

train predictions [3.136      3.13599986 3.17859671 3.1302802  3.15109469 3.23476549
 3.19110574 3.33631802 3.16437921]  
 test predictions [3.17323667]
Train original values [3.136, 3.21, 3.086, 3.136, 3.498, 3.2019999999999995, 3.6480000000000006, 3.036, 2.8325] 
 Test original values 3.2275
MSE train data:  0.06050964997371952
MSE test data:  0.002944509147711605


# Training Loop to find best overall params

In [ ]:
best_params_per_semester = {}
min_mse_per_semester = {}

# Loop through each semester
optimize = False
if optimize == True:
    for semester in range(2,3):
        df = dfs[semester]
        test = tests[semester]
        course_list = courses[semester]
        prediction_index = prediction_indices[semester]

        best_params = None
        min_combined_mse = np.inf

        print("Now at semester ", semester)

        # Iterate over different ARIMA orders
        for p in range(1, 3):
            for d in range(1, 3):
                for q in range(1, 3):
                    total_train_mse = 0
                    total_test_mse = 0
                    valid_courses = 0

                    # Iterate over each course
                    for course_name in course_list:
                        try:
                            # Extract the training and testing series for this course
                            train_row = df.loc[course_name].tolist()
                            
                            # Fit the ARIMA model to this course's training data
                            model = ARIMA(train_row, order=(p, d, q)).fit()

                            # Predict 
                            train_pred = model.predict()
                            train_pred[0] = train_row[0] # fill in first value
                            

                            # Predict next step
                            prediction = model.predict(prediction_index) # no need for start end

                            # Compute MSEs
                            test_mse = mean_squared_error([test.loc[course_name]], prediction)
                            train_mse = mean_squared_error(train_row, train_pred)

                            # add errors
                            total_train_mse += train_mse
                            total_test_mse += test_mse
                            valid_courses += 1

                        except:
                            continue  # Skip if the model fails to fit

                    # Calculate combined MSE and update the best parameters if necessary
                    if valid_courses > 0:
                        combined_mse = total_train_mse + total_test_mse
                        if combined_mse < min_combined_mse:
                            min_combined_mse = combined_mse
                            best_params = (p, d, q)

        # Store the best parameters and combined MSE for this semester
        best_params_per_semester[semester] = best_params
        min_mse_per_semester[semester] = min_combined_mse

    # best_params_per_semester now holds the optimal (p, d, q) for each semester


In [ ]:
best_params_per_semester

{}